---
title: Imperative Programming
skip_execution: true
---

In [ ]:
from __init__ import *

In [ ]:
if not input('Load JupyterAI? [Y/n]').lower()=='n':
    %reload_ext jupyter_ai

## Motivation

It is time to learn to compose a program from its ingredients. [*Imperative programming*](https://en.wikipedia.org/wiki/Imperative_programming) is a programming paradigm that uses the flow of execution of statements, also known as the [control flow](https://en.wikipedia.org/wiki/Control_flow), to change a program's state for the desire computation.

### GCD

To motivate the concept, recall the program for computing the GCD in [Lecture 2](../Lecture2/Elements_of_Computations.ipynb#code_gcd1):

```cpp
{
  b = b % a;
  int c = b;
  b = a;
  a = c;
}
```

where `a` and `b` are the non-zero integers input as follows:

In [ ]:
%%cpp
int a=2*3*4, b=3*4*5;
cout << format("gcd({}, {})=?\n", a, b);

Using control flow statements, we can improve the program to conditionally stop calculating the GCD when it is obtained as the absolute value of `b`, i.e., when `a` becomes `0`:

::::{code} cpp
:label: code_gcd2
:caption: Conditionally compute the GCD of `a` and `b`.
:linenos:
if (a) { // execute if a is non-zero
  b = b % a;
  int c = b;
  b = a;
  a = c;
} else if (b<0) b=-b; // get the absolute value of `b` as the GCD
::::

See if you can crash it by running the program too many times!

In [ ]:
%%cpp
if (a) {
    b = b % a;
    int c = b;
    b = a;
    a = c;
    cout << format("gcd({}, {})\n", a, b); // intermediate answer
} else if (b<0) b=-b;
else cout << b << "\n"; // final answer

We can also free ourselves from repeating the execution manually by writing an *iteration* or a *loop* using control flow statements.

::::{code} cpp
:label: code_gcd3
:caption: Iterative computation of the GCD of `a` and `b`.
:linenos:
int c;
while (a) {  // execute repeatedly as long as a is non-zero
    b = b % a;
    c = b;
    b = a;
    a = c;
}
if (b<0) b=-b;
::::

The program returns the GCD in one go:

In [ ]:
%%cpp
int a=2*3*4, b=3*4*5; // input
cout << format("gcd({}, {})=", a, b);

int c;
while (a) {  // execute repeatedly as long as a is non-zero
    b = b % a;
    c = b;
    b = a;
    a = c;
}
if (b<0) b=-b;
cout << b << "\n"; // final answer

### Inverse Square Root

Recall the update rule of the inverse square root in [Lecture 2](../Lecture2/Elements_of_Computations.ipynb#code_fast_inv_sqrt1_update):

```cpp
y = y * (1.5 - (x*0.5 * y * y));
```

where `x` is the input, and `y` is the initial guess:

In [ ]:
%%cpp
double x = 10./3;  // input
cout << format("rsqrt({})=?\n", x);

auto i = *reinterpret_cast<int64_t *>(&x);
i = 0x5fe6eb50c7b537a9 - (i >> 1);
auto y = *reinterpret_cast<double *>(&i);
cout << format("Initial guess: {}\n", y);  // initial guess

We can improve the program to check whether further execution is required:

::::{code} cpp
:label: code_fast_inv_sqrt2
:caption: Conditional update of a guess `y` of the inverse square root of `x`.
:linenos:
y = (y+1/y/x)/2;   // update the answer

if (auto x_=1/y/y, 
    gap=(x>x_)?(x-x_):(x_-x), 
    abs_x=(x>0?x:-x), 
    abs_x_=(x_>0?x_:-x_); 
    y<0 || x_!=x && gap > 1e-9*(abs_x>abs_x_?abs_x:abs_x_)) { // inaccurate?
    cout << format("Run again to improve the accuracy.");
}
::::

Observe that the program prompts the user to run it again when the answer is inaccurate:

In [ ]:
%%cpp
y = y * (1.5 - (x*0.5 * y * y));

if (auto x_=1/y/y, 
    gap=(x>x_)?(x-x_):(x_-x), 
    abs_x=(x>0?x:-x), 
    abs_x_=(x_>0?x_:-x_); 
    y<0 || x_!=x && gap > 1e-9*(abs_x>abs_x_?abs_x:abs_x_)) {
    cout << format("An improved answer: {}\n", y);
    cout << format("Run again to improve the accuracy.");
} else cout << format("The final answer: {}.\n", y);

The repeated runs can be automated and optimized with the following code:

::::{code} cpp
:label: code_fast_inv_sqrt3
:caption: Iterative update of a guess `y` of the inverse square root of `x`.
:linenos:
constexpr auto threehalfs = 1.5, rel_tol = 1e-9; // for compiler optimization
const auto x2 = x*0.5;                           // avoid redundant calculations
double x_, gap, abs_x, abs_x_;
do {
    y = y * (threehalfs - (x2 * y * y));
    x_=1/y/y; gap=(x>x_)?(x-x_):(x_-x); abs_x=(x>0?x:-x); abs_x_=(x_>0?x_:-x_);
} while (y<0 || x_!=x && gap > rel_tol*(abs_x>abs_x_?abs_x:abs_x_)); // repeat if inaccurate
::::

This gives an accurate answer immediately:

In [ ]:
%%cpp
double x = 10./3;  // input

auto i = *reinterpret_cast<int64_t *>(&x);
i = 0x5fe6eb50c7b537a9 - (i >> 1);
auto y = *reinterpret_cast<double *>(&i);

constexpr auto threehalfs = 1.5, rel_tol = 1e-9;
const auto x2 = x*0.5;
double x_, gap, abs_x, abs_x_;
do {
    y = y * (threehalfs - (x2 * y * y));
    x_=1/y/y; gap=(x>x_)?(x-x_):(x_-x); abs_x=(x>0?x:-x); abs_x_=(x_>0?x_:-x_);
} while (y<0 || x_!=x && gap > rel_tol*(abs_x>abs_x_?abs_x:abs_x_));
cout << format("rsqrt({})={}.\n", x, y); // final answer

### Undecimal to decimal

Consider a new problem of converting an undecimal number to a decimal number.

::::{prf:definition} Undecimal
:label: def:undecimal

An [undecimal number](https://en.wikipedia.org/wiki/Undecimal) is a base 11 number system that uses digits 0-9, along with `X` or `x`, to represent the decimal value 10 similar to the Roman numeral X.

::::

The following is an implementation that converts a undecimal number represented by a C string to an unsigned integer:

::::{code} cpp
:label: code_undecimal1
:caption: Computing the integer value of an undecimal C string `s`.
:linenos:
unsigned long value=0;
constexpr auto m1=-1uL, m2=m1/11, m3=m1-10;
for (size_t i=0, length=strlen(s); i < length; ++i) {
    value *= value<=m2 ? 11 : throw runtime_error("The value is too big.");
    switch (unsigned char c=s[i]) {   // condition in the form of a character declaration
    case 'X':                // case label X
        [[fallthrough]];     // attribute to silent warning
    case 'x':
        value<=m3 ? value+=10 : throw runtime_error("The value is too big.");
        break;               // terminate the switch statement
    default:                 // default label
        if ((c=s[i]-'0')>9) throw runtime_error("Invalid character found.");
        value<=m1-c? value+=c : throw runtime_error("The value is too big.");
    }
}
::::

Control flow statements allow the program to handle undecimal strings of any length, limited only by the maximum integer value.

In [ ]:
%%cpp
auto s="1X2X3x";  // input undecimal string

unsigned long value=0;
constexpr auto m1=-1uL, m2=m1/11, m3=m1-10;
for (size_t i=0, length=strlen(s); i < length; ++i) {
    value *= value<=m2 ? 11 : throw runtime_error("The value is too big.");
    switch (unsigned char c=s[i]) {
    case 'X':
        [[fallthrough]];
    case 'x':
        value<=m3 ? value+=10 : throw runtime_error("The value is too big.");
        break;
    default:
        if ((c=s[i]-'0')>9) throw runtime_error("Invalid character found.");
        value<=m1-c? value+=c : throw runtime_error("The value is too big.");
    }
}
cout << format("Decimal value of the undecimal {} is {}.\n", s, value);

### Distinct Prime Factors

Finally, consider the problem of computing the prime factors:

::::{prf:definition} Prime factor
:label: def:prime_factor

A [prime number](https://en.wikipedia.org/wiki/Prime_number#Definition_and_examples) is a number greater than 1 and cannot be written as the product of two smaller natural numbers.
A prime factor $p$ of a positive integer $n$ is a divisor of $n$ that is a prime number.

::::

The following is an implementation:

::::{code} cpp
:label: code_prime_factor1
:caption: Distinct prime factors of an unsigned integer `n`
:linenos:                   // input
for (auto p=2; p<=n; p++) {     // check candidate factors from 2 to n
    if (n%p) continue;          // continue to the next factor
    bool isprime=true;          // a flag to keep track of the primality
    for (unsigned long d=2L; d*d<=p; d++)  // check candidate divisors
        if (p%d==0) {           // divisor found for p
            isprime=false;
            break;              // break to skip the test of other divisors
        }
    if (isprime) cout << p << '\n'; // prime factor found
} 
::::

Observe that the program prints the distinct prime factors of a positive integer `n` line by line in ascending order, regardless of how many prime factors are there:

In [ ]:
%%cpp
unsigned long n = 100;  // input
cout << format("Distinct prime factors of {} in ascending order:\n", n);
for (unsigned long p=2; p<=n; p++) {
    if (n%p) continue;
    bool isprime=true;
    for (unsigned long d=2L; d*d<=p; d++)
        if (p%d==0) {
            isprime=false;
            break;
        }
    if (isprime) cout << p << '\n';
} 

Imperative programming focuses on using different control flow statements to make decisions and repeat actions in response to different conditions. In the following sections, we will learn how to write conditions and control flow statements.

## Condition

### Boolean Value

Control flow statements determine the flow of execution based on the program state, often summarized as a condition that can be regarded as true or false. Such a condition is said to take a boolean value, named after [George Boole](https://en.wikipedia.org/wiki/George_Boole).

In [ ]:
%%cpp
true

In [ ]:
%%cpp
false

`bool` is a categorical type consisting of only the two values `true` and `false`. For instance, [](#code_prime_factor1) declares a boolean variable `isprime`, which is initialized to `true`, may be updated to `false` later, and is used as a condition to decide whether to print the factor.
```cpp
...
    bool isprime=true;          // a flag to keep track of the primality
    ...
            isprime=false;      // i is not prime
            ...
    if (isprime) ...            // prime factor found
...
```

Theoretically, a boolean value only requires $1$ bit of storage, but C++ stores a boolean value with 1 byte:

In [ ]:
%%cpp
sizeof(bool)

In [ ]:
%%ai
Why C++ uses one byte instead of one bit to store a boolean value? Can I use
just one bit to store a boolean value instead?

Similar to Python, C++ regards `true` as `1`, and `false` as `0`:

In [ ]:
%%cpp
true == 1 && false == 0

In [ ]:
True == 1 and False == 0

We can even apply arithmetic operations on boolean values:

In [ ]:
%%cpp
true + true > true && false * true == false

In [ ]:
True + True, False * True

### Boolean Expression

A simple way to turn the program state to a boolean value is to apply [comparison operators](https://en.cppreference.com/w/cpp/language/operator_comparison.html) such as

>  `<`, `<=`, `>`, `>=`, `<=>`, `==`, `!=`, `<=>`

to numeric values to form a boolean expression, and then combine boolean expressions into a compound boolean expression using [logical operators](https://en.cppreference.com/w/cpp/language/operator_logical.html) such as

1. negation: `!`
2. logical AND: `&&`
3. logical OR: `||`.

Unlike C++, Python uses more expressive names for the logical operators: `not` vs `!`, `and` vs `&&`, `or` vs `||`. In particular, `1 != 0` can be rewritten as `!(1==0)` in C++ but not Python.

Similar to Python, C++ regards any non-zero numeric value as `true` in boolean expressions. For instance, the following checks if all the variables are zero:

In [ ]:
%%cpp
unsigned long x, y, z;
!(x + y + z)

Even though `x + y + z` is not boolean, C++ contextually converts it to boolean since the logical operators expect a boolean context for their operands. Contextual conversion is one of the implicit conversions in C++ that follows [the following rule](https://en.cppreference.com/w/cpp/language/implicit_conversion.html#Boolean_conversions).

> The value zero (for integral, floating-point, and unscoped enumeration) and the [null pointer](https://en.cppreference.com/w/cpp/language/nullptr.html) and the null pointer-to-member values become false. All other values become true.

In [ ]:
%%ai
Explain very briefly what is a null pointer-to-member value in C++?

::::{exercise}
:label: ex:all-zero1

Explain why the following is different from the above check for all zero even if `x`, `y`, and `z` are non-negative.

```cpp
x + y + z != true
```

:::{hint}
:class: dropdown

Try

```cpp
unsigned long x=1, y=1, z;
x + y + z != true
```

:::


::::

YOUR ANSWER HERE

Unlike Python, an empty C string `""` is regarded as `true` but the null character `'\0'` is regarded as `false`:

In [ ]:
%%cpp
auto s="";
auto c=s[0];
s && !c

In [ ]:
not ("" and not "\0")

In [ ]:
%%cpp
unsigned long x, y, z;
!(x || y || z)

::::{exercise}
:label: ex:contextual_conversion

Explain why  an empty C string `""` is regarded as `true` but the null character `'\0'` is regarded as `false`?

::::

YOUR ANSWER HERE

A better check for all-zero is:

In [ ]:
%%cpp
unsigned long x, y, z;
!(x || y || z)

A important benefit of using logical operator `||` in `!(x || y || z)` vs the addition `+` in `!(x + y + z)` is the [short-circuit evaluation](https://en.wikipedia.org/wiki/Short-circuit_evaluation) of logical expressions. To show this, run the following slightly modified code:

In [ ]:
%%cpp
unsigned long x, y, z;
!(++x || ++y || ++z)

In [ ]:
%%cpp
cout << format("x={}, y={}, z={}\n", x, y, z);

`y` and `z` are not incremented because `++x` already evaluates to true, and so the execution of `++y` and `++z` are skipped as their values do not change the overall value of the boolean expression. In comparison, there is no short-circuiting for the addition operations:

In [ ]:
%%cpp
int x, y, z;
!(++x + ++y + ++z)

In [ ]:
%%cpp
cout << format("x={}, y={}, z={}\n", x, y, z);

Python also has short-circuit evaluation, but it returns the value of an operand in its original type without contextual conversion to boolean:

In [ ]:
a = b = 0
a and b%a

In [ ]:
a = 3
b = 5
a and b%a

In contrast, C++'s logical operations return a boolean as the operands are contextually converted to boolean:

In [ ]:
%%cpp
int a, b;
a && b%a

In [ ]:
%%cpp
int a=3, b=5;
a && b%a

### Categories

It is possible to express a condition that takes multiple values. For instance, while most of the comparison operations evaluate to a boolean value, there is an exception—the [three-way comparison operator](https://en.cppreference.com/w/cpp/language/default_comparisons.html#Three-way_comparison) `<=>`, also known as the *spaceship operator*, which returns a comparison category:

In [ ]:
%%cpp
1 <=> 1

[`std::strong_ordering`](https://en.cppreference.com/w/cpp/utility/compare/strong_ordering) from `<compare>` is another categorical type similar to boolean, but with three possible categories below.[^ordering]

- `std::strong_ordering::equal`: The operands are equal.
- `std::strong_ordering::less`: The first operand is less than the second.
- `std::strong_ordering::greater`: The first operand is greater than the second.

[^ordering]: The strong ordering in C++ implements the [strict total ordering](https://en.wikipedia.org/wiki/Total_order#Strict_and_non-strict_total_orders) in order theory. There are two other more general category types:  and  implement respectively the more general 
    - `std::weak_ordering`: Non-strict ordering, which allows distinct items to be equivalent, e.g., in modular arithmetics.
    - `std::partial_ordering`: Partial ordering, which allows distinct items to be incomparable, e.g., in comparing sets.

In [ ]:
%%cpp
auto order = (1 <=> 1);
order == strong_ordering::equal

The check for equality can also be simplified to:

In [ ]:
%%cpp
order == 0

The check for the other two possible categories are as follows:

In [ ]:
%%cpp
auto order = (1 <=> 2);
order == strong_ordering::less && order < 0

In [ ]:
%%cpp
auto order = (2 <=> 1);
(2 <=> 1) == strong_ordering::greater && order > 0

In [ ]:
%%ai
What is the motivation for <=> in C++, given that one can use other comparison
operators to the same effect?

We can define custom categorical type using the [`enum`](https://en.cppreference.com/w/cpp/language/enum.html) declaration. E.g., the following declares the values for [three-valued logic (TVL)](https://en.wikipedia.org/wiki/Three-valued_logic):

In [ ]:
%%cpp
enum TVL {
    UNKNOWN=-1,
    FALSE=0,
    TRUE=1
};
TVL a=TRUE, b=FALSE, c=UNKNOWN;
cout << a << ' ' << b << ' ' << c << '\n';
a==true && b==false

::::{caution}

An `enum` declaration is a simple statement that should end with a semicolon (`;`). Specifically, the braces in the `enum` declaration do not define a compound statement; they merely enclose the list of enumerators.

::::

Manual assignments to integer values are optional, but they ensure that the integer values of `TRUE` and `FALSE` aligns with those of the corresponding boolean values `true` and `false`. Indeed, even the name of the enumeration is optional, as illustrated by the following *unnamed enumeration*:

In [ ]:
%%cpp
enum {     // unnamed
    RED,   // default value 0
    BLUE,  // default value 1
    GREEN  // default value 2
};
auto a=RED, b=BLUE, c=GREEN;
cout << a << ' ' << b << ' ' << c << '\n';
a

Unlike `std::strong_ordering`, the above enumerations are unscoped, which can lead to name conflicts if multiple such enumerations contain a value with the same name.

In [ ]:
%%ai
Give a concrete example of how unscoped enumerations in C++ can cause name 
conflicts.

To resolve the issue, C++11 introduces scoped enumerations with `enum class` and `enum class`, such as:

In [ ]:
%%cpp
enum class TVL {
    UNKNOWN=-1, U=UNKNOWN,        // short aliases
    FALSE=0, F=FALSE,
    TRUE=1, T=TRUE
}
TVL a=TVL::T, b=TVL::F, c=TVL::U;  // qualification `TVL::` required
a

The above also added short aliases `U`, `F`, and `T` without worry of name conflicts because the qualification `TVL::` is required to access the enumerators.

However, the following code will fail because scoped enumerations are not implicitly converted to integers, which is intended to ensure type safety:

```cpp
cout << a << ' ' << b << ' ' << c << '\n';
a==true && b==false
```

In [ ]:
%%cpp
static_cast<int>(TVL::TRUE)

In [ ]:
%%ai
Explain briefly how to properly define three-valued logic data type in C++ so 
that it works in boolean expressions?

## Conditional

### `if` statement

Now that we know how to specify a condition, we can begin to write programs with control flow statements.

The  [`if` statements](https://en.cppreference.com/w/cpp/language/if.html) is a control flow statement that executes a block of code based on whether a condition is true. For instance, in [](#code_gcd2), there are two nested `if` statements:

```cpp
if (a) {
    ... // executed when `a` is true
} else if (b<0) ... // executed when `a` is false but `b<0` is true
```

The first `if` statement executes a compound statement if the condition is true. Otherwise, the control goes to its `else` clause, which consists of a simple `if` statement without an `else` clause.

Unlike Python, the condition for the `if` statement must be enclosed by parentheses `( ... )`, and there is no `elif` keyword to compress `else if`:

In [ ]:
a = b = 0
if a:
    ...
elif b<0: ...

::::{exercise}
:label: ex:gcd_if

Explain the purposes of each of the two `if` statements in [](#code_gcd2).

::::

YOUR ANSWER HERE

[](#code_fast_inv_sqrt2) also has an `if` statement:

```cpp
...
if (auto x_=1/y/y, 
    gap=(x>x_)?(x-x_):(x_-x), 
    abs_x=(x>0?x:-x), 
    abs_x_=(x_>0?x_:-x_); 
    y<0 || x_!=x && gap > 1e-9*(abs_x>abs_x_?abs_x:abs_x_)) { // inaccurate?
    ... //
}  // 
```

Different from the previous `if` statements, there is an [initialization statement](https://en.cppreference.com/w/cpp/language/if.html#if_statements_with_initializer) in front of the condition:

```cpp
...
if (auto x_=1/y/y, 
    gap=(x>x_)?(x-x_):(x_-x), 
    abs_x=(x>0?x:-x), 
    abs_x_=(x_>0?x_:-x_);
    ...
```

The initializer define variables local to the scope of the `if` statement. Hence, repeating the execution does not re-declare the variables in the global scope.

What does the condition mean, however?

```cpp
    ...
    y<0 || x_!=x && gap > 1e-9*(abs_x>abs_x_?abs_x:abs_x_)) { // inaccurate?
...
```

Due to the limited precision of floating point numbers, the accuracy, or the closeness of two floating point numbers, say $x$ and $x'$, should be determined with some tolerance.

::::{prf:definition} closeness
:label: def:isclose

Two floating point numbers `x` and `x'` are said to be close with respect to an absolute tolerance $\delta_{\text{abs}}\geq 0$ or a relative tolerence $\delta_{\text{rel}}\geq 0$ when the following condition is satisfied:

$$
\begin{align}
\lvert x-x' \rvert &\leq \max\Set{\delta_{\text{abs}}, \delta_{\text{rel}}\max \Set{\lvert x\rvert, \lvert x'\rvert}}.
\end{align}
$$ (eq:isclose)

::::

In python, [](#eq:isclose) is implemented by the function `isclose` from the `math` module:

In [ ]:
from math import isclose
help(isclose)

In [ ]:
isclose(100, 90, rel_tol=0.1)  # because (100-90) == 0.1*100

In [ ]:
isclose(100, 89, rel_tol=0.1)  # because (100-89) > 0.1*100

In [ ]:
isclose(100, 90, abs_tol=10)    # because (100-90) == 10

In [ ]:
isclose(100, 89, abs_tol=10)    # because (100-89) > 10

`-inf`, `inf` and `NaN` behave similarly to the IEEE 754 Standard:

In [ ]:
inf = float('inf')
nan = float('NaN')
isclose(inf, inf) and isclose(-inf, -inf) and not isclose(nan, nan) 

Although there is no `isclose` function readily available in C++, it is not difficult to implement it. [](#code_fast_inv_sqrt2) implements it with the help of the [*conditional/tenary operator*](https://en.cppreference.com/w/cpp/language/operator_other.html#Conditional_operator):
```cpp
x>0 ? x : -x
```
The above is analogous to the conditional expression `x if x>0 else -x` in Python, which evaluates to the absolute value of `x`.

::::{exercise}
:label: ex:inv_sqrt_if

Explain how the `if` statement in [](#code_fast_inv_sqrt2) decides whether the answer is inaccurate. 

:::{caution}

Remember to explain the purpose of `y<0 || x_!=x && ...` as well.

:::

::::

YOUR ANSWER HERE

Unlike the conditional expression in Python, the conditional operator in C++ has to decide on the resulting data type based on the types of the operands, since C++ is statically typed.

In [ ]:
%%cpp
true? 'a': 1L                           // why returns a `long`

In [ ]:
%%cpp
false? 'a': false                       // why returns an `int`

As another example, the conditional expression in [](#code_undecimal1)

```cpp
value *= value<=m2 ? 11 : throw runtime_error("The value is too big.");
```

increases `value` by a factor of `11` only if there is no overflow. Otherwise, the [`throw` expression](https://en.cppreference.com/w/cpp/language/throw.html#throw_expressions) will raise a runtime error and transfers the control to an exception handler, and if not handled, exits the program with an error message saying that, "The value is too big."[^throw] The assignment works because the conditional expression has type `int` even though the type of the `throw` expression is [`void`](https://en.cppreference.com/w/cpp/language/types.html#void):

[^throw]: `throw` in C++ is similar to `raise` in Python except that `raise` is used in a statement instead of an expression.

In [ ]:
%%cpp
true? 11 : throw

In constrast, the following code fails because of incompatible operand types.

```cpp
value<=m2 ? 11 : cerr << "The value is too big.";
```

Unlike `throw`, the expression above is not void, but rather, the same type as that of `cerr` (to allow the operator `<<` to be chained.) This case is not handled by the conditional operator, resulting in an error.

In [ ]:
%%ai
Explain very briefly how the resulting type of a conditional operator in C++ is
determined. Give concrete examples.

### `switch` statement

Other than the `if` statement, C++ provides another conditional statement called the [`switch` statement](https://en.cppreference.com/w/cpp/language/switch.html), which acts like the `match` statement in Python to switch execution among possibly more than two cases.

For instance, [](#code_undecimal1) uses the switch statement to convert a digit in a undecimal string to an integer value:

```cpp
...
    switch (unsigned char c=s[i]) {   // condition in the form of a character declaration
    case 'X':                // case label X
        [[fallthrough]];     // attribute to silent warning
    case 'x':
        ...
        break;               // terminate the switch statement
    default:                 // default label
        ...
    }
...
```

Similar to the `if` statement, the condition in a `switch` statement is 

- enclosed in parentheses and,
- optionally, follows an initialization of any number of variables of the same type.

However, unlike the `if` statement, where the condition is expected to be a boolean expression, the `switch` statement allows the condition to be of
- an integral type (such as `int`, `char`, `short`, `long`),
- an enumeration type (such as `enum`, `enum class`, `enum class`), or
- a class type that can be implicitly converted to an integral or enumeration type.

The condition can even be a variable declaration like `(auto c=s[i])` above.

The body of the `switch` statement is typically a compound statement containing 

- several `case` labels, and
- optionally a `default` label

When a switch statement is executed:

1. The control is passed to the first `case` label that matches the value of the condition.
2. If no such match exists, the control is passed to the `default` label. A [`break` statement](https://en.cppreference.com/w/cpp/language/break.html) can be used to terminate the `switch` statement.
3. Otherwise, the control will continue to flow to other cases after the matched case. Such a fall through may trigger a warning, but the attribute [`[[fallthrough]]`](https://en.cppreference.com/w/cpp/language/attributes/fallthrough.html) introduced in C++17 can be used to silent the warning.

::::{exercise}
:label: ex:switch

Explain how the `switch` statement in [](#code_undecimal1) converts a digit in the undecimal string to its decimal value.

::::

YOUR ANSWER HERE

Another way to write the `switch` statement is as follows:

In [ ]:
%%cpp
auto s="1X2X3x";  // input undecimal string

unsigned long value;
unsigned char c;
constexpr auto m=-1uL;
for (unsigned long i=0, length=strlen(s); i < length; ++i) {
    switch (s[i]) {
    case '0': c = 0; break;
    case '1': c = 1; break;
    case '2': c = 2; break;
    case '3': c = 3; break;
    case '4': c = 4; break;
    case '5': c = 5; break;
    case '6': c = 6; break;
    case '7': c = 7; break;
    case '8': c = 8; break;
    case '9': c = 9; break;
    case 'X': [[fallthrough]];
    case 'x': c = 10; break;
    default: throw runtime_error("Invalid character found.");
    }
    value<=(m-c)/11 ? value=value*11+c : throw runtime_error("The value is too big.");
}
cout << format("Decimal value of the undecimal {} is {}.\n", s, value);

::::{exercise}
:label: ex:undecimal_alt

Is the above program more efficient than [](#code_undecimal1) in coverting an undecimal string of all `9`'s such as

> "999999999999999999"

::::

YOUR ANSWER HERE

In [ ]:
%%cpp
auto s="1X2X3x";  // input undecimal string

unsigned long value=0;
constexpr auto m1=-1uL, m2=m1/11, m3=m1-10;
for (size_t i=0, length=strlen(s); i < length; ++i) {
    value<=m2 ? value*=11 : throw runtime_error("The value is too big.");
    switch (unsigned char c=s[i]) {
    case 'X':
        [[fallthrough]];
    case 'x':
        value<=m3 ? value+=10 : throw runtime_error("The value is too big.");
        break;
    default:
        if ((c=s[i]-'0')>9) throw runtime_error("Invalid character found.");
        value<=m1-c? value+=c : throw runtime_error("The value is too big.");
    }
}
cout << format("Decimal value of the undecimal {} is {}.\n", s, value);

To show how the `switch` statement can be applied to enumerations, recall the TVL enumeration:

In [ ]:
%%cpp
enum class TVL {
    UNKNOWN=-1, U=UNKNOWN,
    FALSE=0, F=FALSE,
    TRUE=1, T=TRUE
}

The following code randomly assign a TVL value to a:

In [ ]:
%%cpp
TVL a;
switch (rand() % 3-1) {
case 0: a=TVL::F; break;
case 1: a=TVL::T; break;
default: a=TVL::U;
}
a

::::{exercise}
:label: ex:rand_TVL

Explain whether we can simplify the above code to

```cpp
TVL a = rand()%3 - 1;
```

::::

YOUR ANSWER HERE

The following code implements the logical NOT operation on the randomly drawn value in `a`:

In [ ]:
%%cpp
TVL b;
switch (a) {
case TVL::F: b=TVL::T; break;
case TVL::T: b=TVL::F; break;
default: b=a;
}
b

::::{exercise}
:label: ex:switch_on_ordering

The following code tries to compares two randomly drawn integers `a` and `b`:

```cpp
int a=rand(), b=rand();
switch (auto order=a<=>b; order) {
case strong_ordering::equal:
    cout << "a == b\n";
    break;
case strong_ordering::less:
    cout << "a < b\n";
    break;
default:
    cout << "a > b\n";
}
```

What is wrong with the code?

::::

YOUR ANSWER HERE

## Iteration

### `while` loop

Consider [](#code_gcd3) for computing the GCD iteratively:

```cpp
int c;
while (a) {  // execute repeatedly as long as a is non-zero
    ...
}
...
```

This is almost the same as its previous version [](#code_gcd2) except that:
1. The `if` keyword is changed to the `while` keyword so the body of the [`while` loop](https://en.cppreference.com/w/cpp/language/while.html) gets executed repeatedly as long as the condition holds, i.e., `a` is non-zero.
2. The declaration `int c;` is moved out of the local scope to avoid repeated declarations.

::::{caution}

A simple line of redundant code in an iteration can be costly since it may run many times.

Consider [](#code_fast_inv_sqrt3) for computing the inverse square root iteratively:
```cpp
...
double x_, gap, abs_x, abs_x_;
do {
    ...
    x_=1/y/y; gap=(x>x_)?(x-x_):(x_-x); abs_x=(x>0?x:-x); abs_x_=(x_>0?x_:-x_);
} while (y<0 || x_!=x && gap > 1e-9*(abs_x>abs_x_?abs_x:abs_x_));  // repeat if inaccurate
```

Instead of a while loop, the code uses a [`do-while`](https://en.cppreference.com/w/c/language/do.html) loop, where the looping condition is checked after the execution of the body. This ensures that the accuracy is checked after the answer is updated.

When compared to the previous version [](#code_fast_inv_sqrt2):

1. The declaration `double x_, gap, abs_x, abs_x_;` has moved out of the local scope because `while`/`do-while` loop does not support initializer in the condition, and even if it does support, the repeated declaration is redundant.
2. The update rule is modified to use the constant `x2` and constant expression `threehalfs` to avoid repeated calculations and speed up the access to the values.
    ```cpp
    constexpr auto threehalfs = 1.5, rel_tol = 1e-9;
    const auto x2 = x*0.5;
    auto x2 = x*0.5;
    double x_, gap, abs_x, abs_x_;
    ...
        y = y * (threehalfs - (x2 * y * y));
        ...
    ```

::::{exercise}
:label: ex:y2

Why not define `auto y2 = y * y` to save some more computations in the update rule?

```cpp
y = y * (threehalfs - (x2 * y2))
```
::::

YOUR ANSWER HERE

### `for` loop

Instead of a `while` loop, [](#code_undecimal1) uses a [`for` loop](https://en.cppreference.com/w/cpp/language/for.html):

```cpp
...
for (size_t i=0, length=strlen(s); i < length; ++i) {
    ...
}
```

This can be rewritten as a while loop:

```cpp
...
{
    size_t i=0, length=strlen(s);
    while (i < length) {
        ...
        ++i;
    }
}
```

::::{caution}

`for` loop is often preferred as it is easier to read and safer to write. E.g., in the above example, it is easier to forget to increment `i` in the `while` loop, resulting in an infinite loop.

::::

C++11 introduced a less error prone [range-based `for` loop](https://en.cppreference.com/w/cpp/language/range-for.html):

In [ ]:
%%cpp
for (int i : {0, 1, 2, 3, 4}) cout << i << '\n';

The `for` loop declares `i` to take a value from the initializer list `{0, 1, 2, 3, 4}` one-by-one for each iteration. This is analogous to the Python code:

In [ ]:
for i in 0, 1, 2, 3, 4:
    print(i)

C++20 also introduced [`<ranges>`](https://en.cppreference.com/w/cpp/ranges.html) that makes the range of items more composable and lightweight:

In [ ]:
%%cpp
for (int i : views::iota(0, 5))
    cout << i << '\n';
views::iota(0, 5)

[`std::views::iota(0, 5)`](https://en.cppreference.com/w/cpp/ranges/iota_view.html) from `<ranges>` specifies the sequence of integers from `0` up to but excluding `5`, like `range` in Python code:

In [ ]:
for i in range(5):
    print(i)
range(5)

Finally, [](#code_prime_factor1) uses a nested `for` loop to print the prime factors:

```cpp
for (auto p=2; p<=n; p++) {     // check candidate factors from 2 to n
    ...
    for (unsigned long d=2L; d*d<=p; d++)  // check candidate divisors
        ...
    ...
}
```

For each number `p` from 2 to `n`, it checks whether `p` is divisible by a number `d` from 2 up to the integer square root of `p`. Since the integer square root of `d`, i.e., when `d*d<=p`.

In the outer loop, it uses the  [`continue` statement](https://en.cppreference.com/w/cpp/language/continue.html) to continue to the next iteration:

```cpp
for (auto p=2; p<=n; p++) {     // check candidate factors from 2 to n
    if (n%p) continue;          // continue to the next factor
    ... // code skipped by `continue`
}
```

This is equivalent to

```cpp
for (auto p=2; p<=n; p++) {     // check candidate factors from 2 to n
    if (n%p) {
        ... // code skipped by `continue`
    }
}
```

::::{exercise}
:label: ex:continue
In [](#code_prime_factor1), why is it okay to continue to the next outer loop under the condition `n%p`?

::::

YOUR ANSWER HERE

In the inner loop, it uses the [`break` statement](https://en.cppreference.com/w/cpp/language/break.html) to terminate the inner loop:

```cpp
    ...
    for (unsigned long d=2L; d*d<=p; d++)  // check candidate divisors
        if (p%d==0) {           // divisor found for `p`
            ...
            break;              // break to skip the test of other divisors
        }
    ... // `break` transfers the control to here
```

::::{caution}

A `break` statement can only terminate the closest enclosing loop or a `switch` statement. An enclosing `if` statement does not count, i.e., the following code will fail：
```cpp
if (true) break;
```
::::

The code also uses a flag `isprime` to keep track of the primality of `p` to decide whether to print it as a prime factor later:

```cpp
    ...
    bool isprime=true;          // a flag to keep track of the primality
    for (unsigned long d=2L; d*d<=p; d++)  // check candidate divisors
        if (p%d==0) {           // divisor found for `p`
            isprime=false;
            break;              // break to skip the test of other divisors
        }
    if (isprime) cout << p << '\n'; // prime factor found
    ...
```

::::{exercise} 
:label: ex:prime_factor

How does the following code improve [](#code_prime_factor1) in computing the prime factors? 

::::

In [ ]:
%%cpp
unsigned long n = 100L;  // input

cout << format("Distinct prime factors of {} in ascending order:\n", n);
for (unsigned long p=2; p<=n; p++) {
    if (n%p) continue;
    bool isprime=true;
    unsigned long a=1L, b=p-1, c;
    while (a<b-1) {
        c=(a+b)/2;
        if (p<c*c) b=c; else a=c;
    }
    for (unsigned long d=a; d>1; d--)
        if (p%d==0) {
            isprime=false;
            break;
        }
    if (isprime) cout << p << '\n';
} 

YOUR ANSWER HERE